# Industrial Process Metrics Forecasting

This notebook aims to develop a simple neural network model for the task. We will use a single model to predict all the 9 metrics (we droped the 2 constant metrics in the last step).

## Goals

- Develop an empirical model of the industrial process using AI.
- Achieve millisecond response times for the model.
- Ensure a maximum error of 0.2% compared to the digital twin.

In [44]:
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
X_train = pd.read_csv('data/processed/X_train.csv')
y_train = pd.read_csv('data/processed/y_train.csv')
X_val = pd.read_csv('data/processed/X_val.csv')
y_val = pd.read_csv('data/processed/y_val.csv')
X_test = pd.read_csv('data/processed/X_test.csv')
y_test = pd.read_csv('data/processed/y_test.csv')

In [46]:
features = ['vazaoVapor', 'pressaoVapor', 'temperaturaVapor',
            'cargaVaporTG1', 'cargaVaporTG2', 'habilitaTG1', 'habilitaTG2']

In [47]:
targets = ['consumoEspecificoTG1_1', 'consumoEspecificoTG1_2',
           'consumoEspecificoTG2_1', 'consumoEspecificoTG2_2',
           'potenciaGeradaTG1_1', 'potenciaGeradaTG1_2',
           'potenciaGeradaTG2_1', 'potenciaGeradaTG2_2',
           'vazaoVaporEscape']

In [48]:
target_zeros_count = (y_train == 0).sum()
target_zeros_count

consumoEspecificoTG1_1    129630
consumoEspecificoTG1_2    129630
consumoEspecificoTG2_1    131166
consumoEspecificoTG2_2    131166
potenciaGeradaTG1_1       129630
potenciaGeradaTG1_2       129630
potenciaGeradaTG2_1       131166
potenciaGeradaTG2_2       131166
vazaoVaporEscape               0
dtype: int64

In [49]:
# Percentage of zeros in the target
target_zeros_count / y_train.shape[0]

consumoEspecificoTG1_1    0.500228
consumoEspecificoTG1_2    0.500228
consumoEspecificoTG2_1    0.506155
consumoEspecificoTG2_2    0.506155
potenciaGeradaTG1_1       0.500228
potenciaGeradaTG1_2       0.500228
potenciaGeradaTG2_1       0.506155
potenciaGeradaTG2_2       0.506155
vazaoVaporEscape          0.000000
dtype: float64

In [50]:
columns_with_zeros = target_zeros_count[target_zeros_count > 0].index

## Creating neural network

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras.models import load_model

# To save the models
import joblib

In [52]:
from sklearn.preprocessing import StandardScaler

In [54]:
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
model_checkpoint_cb = ModelCheckpoint('./models/dense_64_32_11_mse_v1.keras', monitor='val_loss', save_best_only=True, verbose=1)

In [55]:
boolean_columns = ['habilitaTG1', 'habilitaTG2']
low_cardinality_columns = [col for col in X_train.columns if X_train[col].nunique() < 15 and col not in boolean_columns]
low_cardinality_columns

[]

In [66]:
new_train = True
model_name = 'dense_64_32_11_mse_v1.1.keras'

try:
    if new_train:
        raise Exception('Forçando novo treinamento')
    model = load_model(f'./models/{model_name}')
    print('Pre-trained model loaded successfully')
except Exception as e:
    print('Could not load model:', e.__repr__())
    model = Sequential()

    model.add(Input(shape=(len(features),)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(targets)))

    model.compile(optimizer='adam', loss='mean_squared_error')

Could not load model: Exception('Forçando novo treinamento')


In [67]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 9)              │           297 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,889 (11.29 KB)

 Trainable params: 2,889 (11.29 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
model_checkpoint_cb = ModelCheckpoint(f'./models/{model_name}', monitor='val_loss', save_best_only=True, verbose=1)

In [69]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Save the scaler
joblib.dump(scaler, './models/scaler_NN.pkl')

In [70]:
if new_train:
    # Set model train seed
    set_random_seed(42)

    history = model.fit(X_train_scaled, y_train, epochs=1000, validation_data=(X_val_scaled, y_val), callbacks=[early_stopping_cb, model_checkpoint_cb])

Epoch 1/1000
8049/8099 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - loss: 1850.6892
Epoch 1: val_loss improved from inf to 30.84571, saving model to ./models/dense_64_32_11_mse_v1.2.keras
8099/8099 ━━━━━━━━━━━━━━━━━━━━ 6s 626us/step - loss: 1841.8365 - val_loss: 30.8457
Epoch 2/1000
8040/8099 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step - loss: 36.6205
Epoch 2: val_loss improved from 30.84571 to 25.84392, saving model to ./models/dense_64_32_11_mse_v1.2.keras
8099/8099 ━━━━━━━━━━━━━━━━━━━━ 5s 617us/step - loss: 36.5381 - val_loss: 25.8439
Epoch 3/1000
8066/8099 ━━━━━━━━━━━━━━━━━━━━ 0s 629us/step - loss: 32.1707
Epoch 3: val_loss improved from 25.84392 to 23.47475, saving model to ./models/dense_64_32_11_mse_v1.2.keras
8099/8099 ━━━━━━━━━━━━━━━━━━━━ 5s 654us/step - loss: 32.1278 - val_loss: 23.4747
Epoch 4/1000
8023/8099 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - loss: 30.1272
Epoch 4: val_loss improved from 23.47475 to 22.19869, saving model to ./models/dense_64_32_11_mse_v1.2.keras
8099/8099 ━━━━━━━━━━━━━━━━

In [71]:
def symetric_mape(A, F):
    """
    Symmetric Mean Absolute Percentage Error (SMAPE) metric.

    Parameters
    ----------
    A : np.array
        Actual values.

    F : np.array
        Forecasted values.

    Returns
    -------
    float
        SMAPE metric value.
    
    """
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

### Reference values for MAPE and SMAPE

| MAPE Value | SMAPE Value | Predictive Performance Evaluation
| :-: | :-: | :-: |
| < 10% | <10% | Highly accurate forecasting |
| 10-20% | 10-20% | Good forecasting |
| 20-50% | 20-50% | Reasonable forecasting |
| >50% | >50% | Inaccurate forecasting |


In [73]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

In [74]:
y_pred = model.predict(X_test_scaled)

450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step


In [75]:
print('Overall metrics:')
print('R2:', r2_score(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))
print('Shifted MAPE:', np.mean(shifted_mape(y_test, y_pred)))
print('Symetric MAPE:', np.mean(symetric_mape(y_test, y_pred)))
print('MSE:', np.mean((y_test - y_pred)**2))

Overall metrics:
R2: 0.9805410490090405
MAPE: 369115749862681.44
Shifted MAPE: 98.65581935093451
Symetric MAPE: 91.15422872398636
MSE: 3.7583571392337385


c:\Users\lucas\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [21]:
smape = np.array([symetric_mape(y_test[target], y_pred[:, i]) for i, target in enumerate(targets)])
mape = np.array([mean_absolute_percentage_error(y_test[target], y_pred[:, i]) for i, target in enumerate(targets)])
r_2 = np.array([r2_score(y_test[target], y_pred[:, i]) for i, target in enumerate(targets)])
mse = np.array([np.mean((y_test[target] - y_pred[:, i])**2) for i, target in enumerate(targets)])

# Create a DataFrame with the MAPE for each target
mape_df = pd.DataFrame({'mape': mape, 'smape': smape, 'shifted mape': shif_mape, 'r2': r_2, 'mse': mse}, index=targets).sort_values('mse')
mape_df

450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


,mape,smape,shifted mape,r2,mse
consumoEspecificoTG1_2,1.486349e+14,100.244385,109.835402,0.999116,0.020197
consumoEspecificoTG2_2,1.904107e+14,102.186390,106.638037,0.999338,0.022514
potenciaGeradaTG2_2,2.479268e+14,104.263931,116.456995,0.996233,0.034226
consumoEspecificoTG2_1,2.170767e+14,103.214952,116.922971,0.990021,0.078247
potenciaGeradaTG1_2,3.259803e+14,100.168433,109.688993,0.999210,0.080970
potenciaGeradaTG2_1,3.418321e+14,103.122653,114.356227,0.998415,0.131863
potenciaGeradaTG1_1,5.135414e+14,101.295703,105.145200,0.998763,0.190267
vazaoVaporEscape,4.732285e-03,0.471885,1.035810,0.998949,5.067501
consumoEspecificoTG1_1,6.353914e+14,104.495129,110.060008,0.919640,15.574207
